In [ ]:
print("Downloading gesture archives...")
!wget "https://rndml-team-cv.obs.ru-moscow-1.hc.sbercloud.ru/datasets/hagrid/hagrid_dataset_new_554800/hagrid_dataset/fist.zip" -O _fist.zip
!wget "https://rndml-team-cv.obs.ru-moscow-1.hc.sbercloud.ru/datasets/hagrid/hagrid_dataset_new_554800/hagrid_dataset/palm.zip" -O _palm.zip
!wget "https://rndml-team-cv.obs.ru-moscow-1.hc.sbercloud.ru/datasets/hagrid/hagrid_dataset_new_554800/hagrid_dataset/like.zip" -O _like.zip
!wget "https://rndml-team-cv.obs.ru-moscow-1.hc.sbercloud.ru/datasets/hagrid/hagrid_dataset_new_554800/hagrid_dataset/dislike.zip" -O _dislike.zip
!wget "https://rndml-team-cv.obs.ru-moscow-1.hc.sbercloud.ru/datasets/hagrid_v2/hagrid_v2_zip/no_gesture.zip" -O _no_gesture.zip # For our 'none' class

print("\nDownloading annotations...")
!wget "https://rndml-team-cv.obs.ru-moscow-1.hc.sbercloud.ru/datasets/hagrid_v2/annotations_with_landmarks/annotations.zip" -O ann_train_val.zip

print("\nAll downloads complete!")

In [14]:
%%writefile unzip.py
import os
import zipfile
import json
import shutil

BASE_PATH = "." 
TEMP_IMAGES_PATH = os.path.join(BASE_PATH, "temp_images")
SORTED_DATA_PATH = os.path.join(BASE_PATH, "data")
ANNOTATIONS_PATH = os.path.join(BASE_PATH, "annotations")

# Create directories inside the current folder
os.makedirs(TEMP_IMAGES_PATH, exist_ok=True)
os.makedirs(SORTED_DATA_PATH, exist_ok=True)
os.makedirs(ANNOTATIONS_PATH, exist_ok=True)

print("Unzipping all downloaded archives...")
all_files_in_dir = os.listdir(BASE_PATH)

for filename in all_files_in_dir:
    if filename.endswith('.zip'):
        print(f"  - Unzipping {filename}...")
        file_path = os.path.join(BASE_PATH, filename)
        
        # Unzip annotations to ANNOTATIONS_PATH
        try: 
            if 'ann_train_val' in filename:
                with zipfile.ZipFile(file_path, 'r') as zip_ref:
                    zip_ref.extractall(ANNOTATIONS_PATH)
            else:
                with zipfile.ZipFile(file_path, 'r') as zip_ref:
                    zip_ref.extractall(TEMP_IMAGES_PATH)
            os.remove(file_path)
            print(f"    - Deleted {filename} to save space.")
        except zipfile.BadZipFile:
            print(f"❌ WARNING: Could not unzip {filename}. It may be corrupt. Skipping.")
print("Unzipping complete.")

Overwriting unzip.py


In [15]:
!python unzip.py

Unzipping all downloaded archives...
Unzipping complete.


In [1]:
!rm -rf data/train

In [7]:
%%writefile sort.py
import os
import shutil
import json

BASE_PATH = "."
SORTED_DATA_PATH = os.path.join(BASE_PATH, "data")
ANNOTATIONS_PATH = os.path.join(BASE_PATH, "annotations")
TEMP_IMAGES_PATH = os.path.join(BASE_PATH, "temp_images")

files_moved_count = 0
print("Starting the sorting process...")

for split in ["train", "val", "test"]:
    split_annotations_dir = os.path.join(ANNOTATIONS_PATH, "annotations", split)
    print(f"\nProcessing '{split}' split...")
    
    if not os.path.exists(split_annotations_dir):
        print(f"  - WARNING: Directory not found: {split_annotations_dir}. Skipping.")
        continue

    json_files = [f for f in os.listdir(split_annotations_dir) if f.endswith('.json')]
    
    for json_file in json_files:
        gesture_name = os.path.splitext(json_file)[0]
        final_dest_folder = os.path.join(SORTED_DATA_PATH, split, gesture_name)
        os.makedirs(final_dest_folder, exist_ok=True)
        
        with open(os.path.join(split_annotations_dir, json_file), 'r') as f:
            annotations = json.load(f)

        for image_id in annotations:
            # Add the .jpg file extension to the image ID to create the full filename
            image_filename = f"{image_id}.jpg"            
            source_image_path = ""
            if gesture_name == 'no_gesture':
                # Use the new filename variable to build the path
                source_image_path = os.path.join(TEMP_IMAGES_PATH, image_filename)
            else:
                # Use the new filename variable to build the path
                source_image_path = os.path.join(TEMP_IMAGES_PATH, gesture_name, image_filename)

            dest_image_path = os.path.join(final_dest_folder, image_filename)

            if os.path.exists(source_image_path):
                shutil.move(source_image_path, dest_image_path)
                files_moved_count += 1
            # else:
            #     print(f"  - WARNING: Source image not found at {source_image_path}")
            #     pass

print(f"\n✅ Sorting complete! Total files moved: {files_moved_count}")

print("\nInitiating cleanup phase...")

if files_moved_count > 0:
    print(f"Cleanup condition met ({files_moved_count} files were moved). Deleting temporary folders.")
    if os.path.exists(TEMP_IMAGES_PATH):
        shutil.rmtree(TEMP_IMAGES_PATH)
    if os.path.exists(ANNOTATIONS_PATH):
        shutil.rmtree(ANNOTATIONS_PATH)
    print("✅ Cleanup finished.")
else:
    print("❌ WARNING: Cleanup condition not met. No files were moved during sorting.")
    print("      The temporary folders will NOT be deleted for debugging.")

Overwriting sort.py


In [9]:
!python sort.py

Starting the sorting process...

Processing 'train' split...

Processing 'val' split...

Processing 'test' split...

✅ Sorting complete! Total files moved: 104661

Initiating cleanup phase...
Cleanup condition met (104661 files were moved). Deleting temporary folders.
✅ Cleanup finished.


In [6]:
%%writefile cleanup.py
import os
import shutil

GESTURES_TO_KEEP = {
    'fist', 
    'like', 
    'no_gesture', 
    'palm'
}

DATA_PATH = "data"

print("Starting Cleanup Script")

if not os.path.isdir(DATA_PATH):
    print(f"Error: Data directory not found at '{DATA_PATH}'. Exiting.")
else:
    # Loop through each split directory (train, val, test)
    for split_name in os.listdir(DATA_PATH):
        split_dir_path = os.path.join(DATA_PATH, split_name)
        
        if os.path.isdir(split_dir_path):
            print(f"\nProcessing directory: {split_dir_path}")
            
            # Loop through each gesture folder in the split directory
            for gesture_name in os.listdir(split_dir_path):
                gesture_folder_path = os.path.join(split_dir_path, gesture_name)
                
                # Check if the folder is actually a directory and NOT in our keep list
                if os.path.isdir(gesture_folder_path) and gesture_name not in GESTURES_TO_KEEP:
                    try:
                        print(f"  - Deleting unwanted folder: {gesture_folder_path}")
                        shutil.rmtree(gesture_folder_path)
                    except OSError as e:
                        print(f"  - Error deleting {gesture_folder_path} : {e.strerror}")

print("\n--- Cleanup Complete ---")

Overwriting cleanup.py


In [7]:
!python cleanup.py

Starting Cleanup Script

Processing directory: data/train
  - Deleting unwanted folder: data/train/dislike

Processing directory: data/test
  - Deleting unwanted folder: data/test/dislike

Processing directory: data/val
  - Deleting unwanted folder: data/val/dislike

--- Cleanup Complete ---


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from tqdm import tqdm
import os
import time
import copy

In [2]:
# setup and configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

data_dir = 'data'
batch_size = 32
num_workers = 5
num_epochs = 20

Using device: cuda


In [3]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(30), 
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4), 
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
data_transforms['test'] = data_transforms['val']

In [4]:
# load the datasets using ImageFolder
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'val', 'test']}

# create the dataloaders
dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True),
    'val': DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True),
    'test': DataLoader(image_datasets['test'], batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)
}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}

class_names = image_datasets['train'].classes
num_classes = len(class_names)
print("Class names found:", class_names)
print(f"{num_classes} classes detected.")

Class names found: ['fist', 'like', 'no_gesture', 'palm']
4 classes detected.


/home/ec2-user/SageMaker/GestureControl/gesturevenv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [5]:
model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)

# freeze all layers first
for param in model.parameters():
    param.requires_grad = False

# Unfreeze the last few convolutional blocks
for param in model.features[-5:].parameters():
    param.requires_grad = True

# Replace the classifier head, making sure its parameters are trainable
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, num_classes)

model = model.to(device)

Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /home/ec2-user/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 243MB/s]


In [6]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam([
    {'params': model.features.parameters(), 'lr': 1e-5}, 
    {'params': model.classifier.parameters(), 'lr': 1e-3}
])

# set up a learning rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
num_epochs = 50 
patience = 5  # number of epochs to wait for improvement before stopping
epochs_no_improve = 0
early_stop = False

start_time = time.time()
best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 10)

    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in tqdm(dataloaders[phase], desc=phase):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]
        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # early stopping
        if phase == 'val':
            # if validation accuracy has improved, save the model and reset the counter
            if epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                epochs_no_improve = 0
            # if validation accuracy has not improved, increment the counter
            else:
                epochs_no_improve += 1
    
    # check if we should stop after each epoch
    if epochs_no_improve >= patience:
        print(f'\nEarly stopping triggered after {patience} epochs with no improvement.')
        early_stop = True
        break  

    scheduler.step()
    print()

time_elapsed = time.time() - start_time
print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
print(f'Best val Acc: {best_acc:4f}')

# load the best model weights found during training
model.load_state_dict(best_model_wts)
print("Best model weights have been loaded.")
save_path = 'gesture_best.pth'
torch.save(model.state_dict(), save_path)
print(f"Model successfully saved to: {save_path}")

Epoch 1/50
----------


train: 100%|██████████| 2249/2249 [1:10:43<00:00,  1.89s/it]


train Loss: 0.6444 Acc: 0.7283


val: 100%|██████████| 288/288 [06:54<00:00,  1.44s/it]


val Loss: 0.2870 Acc: 0.8898

Epoch 2/50
----------


...

val Loss: 0.1325 Acc: 0.9513

Epoch 24/50
----------


train: 100%|██████████| 2249/2249 [21:32<00:00,  1.74it/s]


train Loss: 0.2173 Acc: 0.9139


val: 100%|██████████| 288/288 [02:38<00:00,  1.81it/s]

val Loss: 0.1318 Acc: 0.9508

Early stopping triggered after 5 epochs with no improvement.
Training complete in 637m 3s
Best val Acc: 0.951304
Best model weights have been loaded.
Model successfully saved to: gesture_best.pth
